In [10]:
import pandas
import configparser
import psycopg2

In [18]:
config = configparser.ConfigParser()
config.read('config.ini')

host = config['myaws']['host']
db = config['myaws']['database']
user = config['myaws']['user']
pwd = config['myaws']['pwsd']

In [45]:
conn = psycopg2.connect(host = host,
                       user = user,
                        password = pwd,
                        dbname = db
                       )
cur = conn.cursor()

In [21]:
table_sql = """
            CREATE TABLE IF NOT EXISTS indeed_gp7
            (
                id SERIAL,
                job_title VARCHAR(200),
                job_company VARCHAR(200),
                job_loc VARCHAR(200),
                job_salary VARCHAR(200),
                job_summary TEXT,
                PRIMARY KEY(id)
            );

            """

In [22]:
cur.execute(table_sql)
conn.commit()

In [50]:
url = 'https://www.indeed.com/jobs?q=intelligence+analyst&start=10'

In [51]:
import urllib.request
response = urllib.request.urlopen(url)
html_data= response.read()
#print(html_data.decode('utf-8'))

In [52]:
!pip install beautifulsoup4


In [53]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_data,'html.parser')
#print (soup)

In [54]:
for table_resultsBody in soup.find_all('table', id = 'resultsBody'):
    pass
    #print(table_resultsBody)

In [55]:
for table_pageContent in table_resultsBody.find_all('table', id = 'pageContent'):
    pass
    #print(table_pageContent)

In [56]:
for td_resultsCol in table_pageContent.find_all('td', id = 'resultsCol'):
    pass
    #print(td_resultsCol)

In [57]:
#  identify the job title, company, ratings, reviews, salary, and summary
for div_row in td_resultsCol.find_all('div', class_='jobsearch-SerpJobCard unifiedRow row result'):

    # find job title
    job_title = None
    job_company  = None
    job_rating = None
    job_loc = None
    job_salary = None
    job_summary = None
    for div_title in div_row.find_all('div', class_ = 'title'):
        job_title = div_title.text.strip().replace("'","_")
    
    for div_dsc in div_row.find_all('div', class_ = 'sjcl'):
        
        #find company name
        for span_company in div_dsc.find_all('span', class_ = 'company'):
            job_company = span_company.text.strip().replace("'","_")
        
        # find location
        for div_loc in div_dsc.find_all('div', class_ = 'location accessible-contrast-color-location'):
            job_loc = div_loc.text.strip().replace("'","_")
    
    # find salary
    for div_salary in div_row.find_all('div',class_ ='salarySnippet'):
        job_salary = div_salary.text.strip().replace("'","_")
    
    #find summary
    for div_summary in div_row.find_all('div', class_ = 'summary'):
        job_summary = div_summary.text.strip().replace("'","_")
        
    # insert into database
    
    sql_insert = """
                insert into indeed_gp7(job_title,job_company,job_loc,job_salary,job_summary)
                values('{}','{}','{}','{}','{}')            
                """.format(job_title,job_company,job_loc,job_salary,job_summary)

    cur.execute(sql_insert)
    conn.commit()

In [58]:
df = pandas.read_sql_query('select * from indeed_gp7', conn)
df[:]

,id,job_title,job_company,job_loc,job_salary,job_summary
0,1,Intelligence Analyst,National Security Agency,"Fort Meade, MD","$47,016 - $74,759 a year",Apply new techniques and develop creative solu...
1,2,Quantitative Analyst / Developer,RT Signal,"New York, NY 10013 (Tribeca area)","$95,000 - $165,000 a year",If you are a Go developer looking for a challe...
2,3,ERP Applications Developer (Epicor),"Ashworth Bros., Inc.","Winchester, VA 22601",None,This may include custom mobile and windows app...
3,4,Transportation Analyst,Lipman Family Farms,"Immokalee, FL 34142",None,"Perform fact based, data driven analysis, and ..."
4,5,H-ISAC Cyber Threat Intelligence Analyst,Perch Security,"Titusville, FL","$50,000 - $75,000 a year",The CTI Analyst will be located in our Securit...
5,6,"Cybersecurity Analyst (DLP) (60-70k, US Citize...",Maveris,"Martinsburg, WV","$60,000 - $70,000 a year",The Cybersecurity Analyst can effectively leve...
6,7,Brand Intelligence Analyst,LookingGlass Cyber Solutions,"Reston, VA 20191",None,Alerting and providing rapid responses to miti...
7,8,Intelligence Specialist (S&T Analyst),US Department of the Army,None,"$88,704 - $115,313 a year",One year of specialized experience equivalent ...
8,9,Intelligence Analyst (Identity Intelligence),"Signature Science, LLC",None,None,Candidate must be willing to travel to Washing...
9,10,Intelligence Specialist (GMI Analyst),US Department of the Army,None,"$88,704 - $115,313 a year",Conducts research and analysis of counterterro...
